About the Dataset:

1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article; could be incomplete
5. label: a label that marks whether the news article is real or fake:
           1: Fake news
           0: real News





Importing the Dependencies

In [ ]:
import numpy as np
import pandas as pd
import re 
# re is regular expression. it is used to search a text in the document
from nltk.corpus import stopwords 
# corpus is main content or important content of a document
# stopwords are those words that do not add much value to our paragraph (articles, wh words)
# nltk stands for natural language tool kit
from nltk.stem.porter import PorterStemmer

# stemming takes the word and removes the suffix of that word and returns the root word
from sklearn.feature_extraction.text import TfidfVectorizer
# this function makes the text into vectors (numbers)
from sklearn.model_selection import train_test_split
# splits data into train and testing data
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [ ]:

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')
german_stop_words = stopwords.words('german')
vect = CountVectorizer(stop_words = german_stop_words) # Now use this in your pipeline


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Data Pre-processing

In [ ]:
# loading the dataset to a pandas DataFrame
news_dataset = pd.read_csv('german.csv')

In [ ]:
news_dataset.shape

(1249, 4)

In [ ]:
# print the first 5 rows of the dataframe
news_dataset.head()

,id,Title,Text,Label
0,773233,Exklusiv! Das geheime WhatsApp-Chat-Protokoll ...,Die Sondierungsgespräche zwischen Union und SP...,1
1,773234,"Trump droht, jeden zu verspeisen, der an seine...",Nun ist es auch medizinisch offiziell bestätig...,1
2,773235,"Soli runter, keine Steuererhöhungen, kein Klim...","Es waren zähe Verhandlungen, doch die Freien D...",1
3,773236,Hat sie eine Affäre? Joachim Sauer glaubt Ange...,Wo treibt sie sich immer bis spät in die Nacht...,1
4,773237,"""Er hat ja nur HALBneger gesagt"": So begründet...",Der Parteivorstand drückt nochmal ein Auge zu:...,1


In [ ]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()

id       0
Title    0
Text     0
Label    0
dtype: int64

In [ ]:
# replacing the null values with empty string
news_dataset = news_dataset.fillna('') #fillna= fill name 

In [ ]:

news_dataset['content'] = news_dataset['Title']

In [ ]:
print(news_dataset['content'])

0       Exklusiv! Das geheime WhatsApp-Chat-Protokoll ...
1       Trump droht, jeden zu verspeisen, der an seine...
2       Soli runter, keine Steuererhöhungen, kein Klim...
3       Hat sie eine Affäre? Joachim Sauer glaubt Ange...
4       "Er hat ja nur HALBneger gesagt": So begründet...
                              ...                        
1244    Bundesverfassungsgericht erklärt Regierung für...
1245    Europaweiter Jubel: Politiker beenden Krise du...
1246    Verfassungsschutz wird künftig von Horde Schim...
1247    Petition, dass Politiker regelmäßig Backpfeife...
1248    Innenministerium lässt Verfassungsschutz vom V...
Name: content, Length: 1249, dtype: object


In [ ]:
news_dataset.shape

(1249, 5)

In [ ]:
# separating the data & label
X = news_dataset.drop(columns='Label', axis=1)
Y = news_dataset['Label']

In [ ]:
print(X)
print(Y)

          id                                              Title  \
0     773233  Exklusiv! Das geheime WhatsApp-Chat-Protokoll ...   
1     773234  Trump droht, jeden zu verspeisen, der an seine...   
2     773235  Soli runter, keine Steuererhöhungen, kein Klim...   
3     773236  Hat sie eine Affäre? Joachim Sauer glaubt Ange...   
4     773237  "Er hat ja nur HALBneger gesagt": So begründet...   
...      ...                                                ...   
1244  773925  Bundesverfassungsgericht erklärt Regierung für...   
1245  773926  Europaweiter Jubel: Politiker beenden Krise du...   
1246  773927  Verfassungsschutz wird künftig von Horde Schim...   
1247  773928  Petition, dass Politiker regelmäßig Backpfeife...   
1248  773929  Innenministerium lässt Verfassungsschutz vom V...   

                                                   Text  \
0     Die Sondierungsgespräche zwischen Union und SP...   
1     Nun ist es auch medizinisch offiziell bestätig...   
2     Es waren zäh

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content) #this removes evething except lowercase and upper case alphabets
    stemmed_content = stemmed_content.lower() #converting everything to lowercase letter
    stemmed_content = stemmed_content.split() #converting to list
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('german')]
    # stemming everything except stopwords
    stemmed_content = ' '.join(stemmed_content) #joining all the words
    return stemmed_content #processed text

In [ ]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [ ]:
print(news_dataset['content'])

0       exklusiv geheim whatsapp chat protokol groko s...
1       trump droht verspeisen geistigen gesundheit zw...
2       soli runter steuererh hungen klimaziel fdp set...
3       aff re joachim sauer glaubt angela merkel albe...
4       ja halbneg gesagt begr ndet afd jen maier blei...
                              ...                        
1244    bundesverfassungsgericht erkl rt regierung f r...
1245    europaweit jubel politik beenden krise effekt ...
1246    verfassungsschutz k nftig hord schimpansen gef...
1247    petit politik regelm ig backpfeifen erhalten e...
1248    innenministerium l sst verfassungsschutz verfa...
Name: content, Length: 1249, dtype: object


In [ ]:
#separating the data and label
X = news_dataset['content'].values
Y = news_dataset['Label'].values

In [ ]:
print(X)

['exklusiv geheim whatsapp chat protokol groko sondierungsgespr che'
 'trump droht verspeisen geistigen gesundheit zweifelt'
 'soli runter steuererh hungen klimaziel fdp setzt sondierungen' ...
 'verfassungsschutz k nftig hord schimpansen gef hrt'
 'petit politik regelm ig backpfeifen erhalten erreicht quorum sekunden'
 'innenministerium l sst verfassungsschutz verfassungsschutz beobachten']


In [ ]:
print(Y)

[1 1 1 ... 1 1 1]


In [ ]:
Y.shape 
# shows rows in Y

(1249,)

In [ ]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [ ]:
print(X)

  (0, 3610)	0.3653694324843706
  (0, 2893)	0.3388289797953309
  (0, 2454)	0.3504226246537418
  (0, 1370)	0.38643577492715064
  (0, 1215)	0.3504226246537418
  (0, 996)	0.3388289797953309
  (0, 584)	0.3293562822109617
  (0, 583)	0.3653694324843706
  (1, 3753)	0.45406460109809293
  (1, 3431)	0.45406460109809293
  (1, 3209)	0.2613340281504413
  (1, 1302)	0.45406460109809293
  (1, 1226)	0.45406460109809293
  (1, 786)	0.32711752910717473
  (2, 3017)	0.38555639259793695
  (2, 2891)	0.36453798921899966
  (2, 2882)	0.38555639259793695
  (2, 2835)	0.29718606838108086
  (2, 2679)	0.38555639259793695
  (2, 1728)	0.36453798921899966
  (2, 1519)	0.36453798921899966
  (2, 1038)	0.2589505906332278
  (3, 2890)	0.34501227308867466
  (3, 2713)	0.34501227308867466
  (3, 2512)	0.25474804871440576
  :	:
  (1245, 1962)	0.377151098697677
  (1245, 1807)	0.33068812540586456
  (1245, 1642)	0.377151098697677
  (1245, 989)	0.3565909054823004
  (1245, 817)	0.377151098697677
  (1245, 330)	0.3420032162436231
  (1246,

Splitting the dataset to training & test data

In [ ]:
# 80% training data and 20% testing data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

Training the Model: Logistic Regression

In [ ]:
model = LogisticRegression()

In [ ]:
# training our model
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation

accuracy score

In [ ]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [ ]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.960960960960961


In [ ]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [ ]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.724


Making a Predictive System

In [ ]:
X_new = X_test[225]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[1]
The news is Fake


In [ ]:
print(Y_test[225])

1
